In [1]:
%pip install requests beautifulsoup4 lxml
%pip install openpyxl xlsxwriter

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [2]:


BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

def get_max_page(prodi_id):
    """Mendapatkan jumlah halaman maksimal untuk prodi tertentu"""
    try:
        url = f"{BASE_URL}/{prodi_id}/1"
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, "html.parser")
        
        pagination = soup.select('a[href*="byprod"]')
        if pagination:
            last_page = 1
            for link in pagination:
                href = link.get('href', '')
                if f'/byprod/{prodi_id}/' in href:
                    page_match = re.search(rf'/byprod/{prodi_id}/(\d+)', href)
                    if page_match:
                        page_num = int(page_match.group(1))
                        last_page = max(last_page, page_num)
            return last_page
        return 1
    except:
        return 1

def pta():
    """Fungsi utama untuk scraping data PTA"""
    start_time = time.time()
    
    data = {
        "judul": [], "abstrak_id": [], "abstrak_en": []
    }
    
    prodi_id = 7
    prodi_name = "Manajemen"
    
    # Get max pages
    max_page = get_max_page(prodi_id)
    
    for j in range(1, max_page + 1):
        try:
            url = f"{BASE_URL}/{prodi_id}/{j}"
            r = requests.get(url, timeout=15)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')
            
            # Process setiap jurnal
            for jurnal in jurnals:
                try:
                    link_keluar = jurnal.select_one('a.gray.button')['href']
                    
                    response = requests.get(link_keluar, timeout=15)
                    soup1 = BeautifulSoup(response.content, "html.parser")
                    isi = soup1.select_one('div#content_journal')
                    
                    if not isi:
                        continue
                    
                    # Extract data
                    judul = isi.select_one('a.title').text.strip()
                    
                    # Extract abstrak
                    paragraf = isi.select('p[align="justify"]')
                    abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                    abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"
                    
                    # Clean abstrak Indonesia
                    if abstrak_id != "N/A" and abstrak_id.upper().startswith('ABSTRAK'):
                        abstrak_id = abstrak_id[7:].strip()
                    
                    # Append data
                    data["judul"].append(judul)
                    data["abstrak_id"].append(abstrak_id)
                    data["abstrak_en"].append(abstrak_en)
                    
                    time.sleep(0.1)
                    
                except:
                    continue
            
        except:
            continue
    
    # Buat DataFrame
    df = pd.DataFrame(data)
    
    # Summary
    print(f"📊 Total data dikumpulkan: {len(df):,}")
    
    return df

# Jalankan scraping
result_df = pta()

# Ambil abstrak Indonesia untuk corpus
corpus = result_df[result_df['abstrak_id'] != 'N/A']['abstrak_id'].tolist()

# Tampilkan dalam bentuk DataFrame (semua data)
import pandas as pd
df_sample = pd.DataFrame({
    'No': range(1, len(corpus) + 1),
    'Abstrak': corpus
})

df_sample

### **1. Pembersihan Teks**

**Pembersihan teks** adalah tahap preprocessing untuk menormalkan data teks agar konsisten dan siap dianalisis. Proses ini meliputi: mengubah teks ke huruf kecil (lowercasing), menghapus angka, menghilangkan tanda baca, dan membersihkan spasi berlebih. Tujuannya adalah mengurangi variasi yang tidak perlu dan memfokuskan analisis pada konten tekstual yang bermakna.

In [3]:
# Pembersihan teks
import re
import pandas as pd

def clean_text(text):
    # Ubah ke huruf kecil
    text = text.lower()
    # Hapus angka
    text = re.sub(r'\d+', '', text)
    # Hapus tanda baca
    text = re.sub(r'[^\w\s]', '', text)
    # Hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Terapkan pembersihan teks
cleaned_corpus = [clean_text(text) for text in corpus]

# Tampilkan hasil pembersihan teks dalam format DataFrame
comparison_df = pd.DataFrame({
    'abstrak_asli': corpus[:10],
    'abstrak_bersih': cleaned_corpus[:10]
})

print("\nHasil Pembersihan Teks:")
comparison_df


Hasil Pembersihan Teks:


,abstrak_asli,abstrak_bersih
0,"Satiyah, Pengaruh Faktor-faktor Pelatihan dan ...",satiyah pengaruh faktorfaktor pelatihan dan pe...
1,Tujuan penelitian ini adalah untuk mengetahui ...,tujuan penelitian ini adalah untuk mengetahui ...
2,,
3,Aplikasi nyata pemanfaatan teknologi informasi...,aplikasi nyata pemanfaatan teknologi informasi...
4,"Penelitian ini menggunakan metode kuantitatif,...",penelitian ini menggunakan metode kuantitatif ...
5,"Aththaariq, Pengaruh Kompetensi Dosen Terhadap...",aththaariq pengaruh kompetensi dosen terhadap ...
6,"Haryono Arifin, Pengaruh Perilaku Konsumen Ter...",haryono arifin pengaruh perilaku konsumen terh...
7,"Dharma Abidin Syah,Kesimpulan: (1) Terdapat pe...",dharma abidin syahkesimpulan terdapat pengaruh...
8,Tujuan penelitian ini adalah untuk mengidentif...,tujuan penelitian ini adalah untuk mengidentif...
9,Hasil dari penelitian ini dari perhitungan Cre...,hasil dari penelitian ini dari perhitungan cre...


### **2. Tokenisasi**

**Tokenisasi** adalah proses memecah teks menjadi unit-unit yang lebih kecil seperti kata atau token. Pada tahap ini, setiap kalimat dalam abstrak dipecah menjadi kata-kata individual menggunakan `word_tokenize` dari NLTK. Contoh: kalimat "Penelitian ini menganalisis data" → ['Penelitian', 'ini', 'menganalisis', 'data']

In [4]:
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd

nltk.download('punkt_tab')

# Tokenisasi untuk PTA
# Buat DataFrame dari corpus yang sudah dibersihkan
pta_df = pd.DataFrame({
    'abstrak_id_clean': cleaned_corpus
})

pta_df["abstrak_id_tokens"] = pta_df["abstrak_id_clean"].apply(word_tokenize)

print("\nPTA (abstrak_id_tokens):")
pta_df[["abstrak_id_clean", "abstrak_id_tokens"]].head(10)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data]   Package punkt_tab is already up-to-date!



PTA (abstrak_id_tokens):


,abstrak_id_clean,abstrak_id_tokens
0,satiyah pengaruh faktorfaktor pelatihan dan pe...,"[satiyah, pengaruh, faktorfaktor, pelatihan, d..."
1,tujuan penelitian ini adalah untuk mengetahui ...,"[tujuan, penelitian, ini, adalah, untuk, menge..."
2,,[]
3,aplikasi nyata pemanfaatan teknologi informasi...,"[aplikasi, nyata, pemanfaatan, teknologi, info..."
4,penelitian ini menggunakan metode kuantitatif ...,"[penelitian, ini, menggunakan, metode, kuantit..."
5,aththaariq pengaruh kompetensi dosen terhadap ...,"[aththaariq, pengaruh, kompetensi, dosen, terh..."
6,haryono arifin pengaruh perilaku konsumen terh...,"[haryono, arifin, pengaruh, perilaku, konsumen..."
7,dharma abidin syahkesimpulan terdapat pengaruh...,"[dharma, abidin, syahkesimpulan, terdapat, pen..."
8,tujuan penelitian ini adalah untuk mengidentif...,"[tujuan, penelitian, ini, adalah, untuk, mengi..."
9,hasil dari penelitian ini dari perhitungan cre...,"[hasil, dari, penelitian, ini, dari, perhitung..."


### **3. Penghapusan Kata Umum (Stop Words)**

**Stop Words** adalah kata-kata umum yang sering muncul dalam bahasa namun tidak memberikan makna signifikan untuk analisis teks, seperti "dan", "atau", "di", "ke", "yang". Penghapusan stopwords bertujuan untuk mengurangi noise dan fokus pada kata-kata yang lebih bermakna untuk analisis. NLTK menyediakan daftar stopwords bahasa Indonesia yang sudah siap pakai.

In [5]:
from nltk.corpus import stopwords
import nltk

# Download stopwords untuk bahasa Indonesia
nltk.download('stopwords')

# Stopwords untuk bahasa Indonesia
stop_words_id = set(stopwords.words('indonesian'))

# Filter stopwords di PTA
pta_df["abstrak_id_filtered"] = pta_df["abstrak_id_tokens"].apply(
    lambda tokens: [word for word in tokens if word not in stop_words_id]
)

print("\nPTA (abstrak_id_filtered):")
pta_df[["abstrak_id_tokens", "abstrak_id_filtered"]].head(10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



PTA (abstrak_id_filtered):


,abstrak_id_tokens,abstrak_id_filtered
0,"[satiyah, pengaruh, faktorfaktor, pelatihan, d...","[satiyah, pengaruh, faktorfaktor, pelatihan, p..."
1,"[tujuan, penelitian, ini, adalah, untuk, menge...","[tujuan, penelitian, persepsi, brand, associat..."
2,[],[]
3,"[aplikasi, nyata, pemanfaatan, teknologi, info...","[aplikasi, nyata, pemanfaatan, teknologi, info..."
4,"[penelitian, ini, menggunakan, metode, kuantit...","[penelitian, metode, kuantitatif, menekankan, ..."
5,"[aththaariq, pengaruh, kompetensi, dosen, terh...","[aththaariq, pengaruh, kompetensi, dosen, kine..."
6,"[haryono, arifin, pengaruh, perilaku, konsumen...","[haryono, arifin, pengaruh, perilaku, konsumen..."
7,"[dharma, abidin, syahkesimpulan, terdapat, pen...","[dharma, abidin, syahkesimpulan, pengaruh, sig..."
8,"[tujuan, penelitian, ini, adalah, untuk, mengi...","[tujuan, penelitian, mengidentifikasi, variabe..."
9,"[hasil, dari, penelitian, ini, dari, perhitung...","[hasil, penelitian, perhitungan, credit, risk,..."


### **4. Stemming dan Lematisasi**

**Stemming** adalah proses mengubah kata-kata menjadi bentuk dasar (root/stem) dengan menghilangkan imbuhan seperti awalan, akhiran, dan sisipan. Contoh: "penelitian" → "teliti", "menganalisis" → "analisis". **Lematisasi** serupa dengan stemming namun menghasilkan kata dasar yang lebih bermakna secara linguistik. Pada kode ini menggunakan library Sastrawi untuk stemming bahasa Indonesia dan TF-IDF untuk mengukur kepentingan kata dalam dokumen.

In [6]:
import Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Test stemming dengan satu dokumen
input_stemm = str(pta_df["abstrak_id_filtered"].iloc[0])
hasil_stemm = stemmer.stem(input_stemm)
print(hasil_stemm)

# Stemming untuk semua dokumen
hasil_stemm = []
for doc in pta_df["abstrak_id_filtered"]:
    stemmed_doc = [stemmer.stem(word) for word in doc]
    hasil_stemm.append(stemmed_doc)

# Konversi hasil stemming ke string untuk TF-IDF
stemmed_texts = [' '.join(doc) for doc in hasil_stemm]

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(stemmed_texts)
print(x)

satiyah pengaruh faktorfaktor latih kembang produktivitas kerja dinas laut ikan bangkal bawah bimbing drahjsanugrahini irawatimm helm buyung auliasstsemmt upaya tingkat produktivitas kerja mudah salah usaha produktivitas tingkat terap program latih kembang sumber daya manusia sdm laksana instansi produktivitas capai tingkat mampu pegawai efektif efisien latih pengembnagan harap pegawai sesuai kebutuhankebutuhan sikap tingkah laku terampil tahu sesuai tuntut ubah latih kembang pegawai dukung cipta suasana kerja kondusif instansi produktivitas kerja tingkat tuju teliti pengaruh faktorfaktor latih kembang produktivitas kerja dinas laut ikan bangkal ukur menganalisa hubung variabel teliti dekat observasional analitik amat langsung responden sebar kuisioner analis teliti teliti populasi responden sampel olah spss versi analis metode statistik metode non probality sampling simple random sampling simpul teliti a faktorfaktor latih kembang beda individu pegawai x hubung analisis jabat x motiva

In [7]:
from nltk import FreqDist
import pandas as pd

# Gabungkan semua kata dari hasil stemming
all_words = []
for doc in hasil_stemm:
    all_words.extend(doc)

fdist = FreqDist(all_words)

frequency_df = pd.DataFrame(
    fdist.most_common(),
    columns=['Word', 'Frequency']
)

frequency_df.to_csv('Management_dataHasilPreprocessing.csv', index=False, encoding='utf-8-sig')
print("File disimpan sebagai Management_dataHasilPreprocessing.csv")

File disimpan sebagai Management_dataHasilPreprocessing.csv


### **5. Word Embeddings dengan CBOW (Continuous Bag of Words)**

**CBOW** adalah salah satu arsitektur Word2Vec yang memprediksi kata target berdasarkan konteks di sekitarnya. Berbeda dengan TF-IDF yang menghasilkan sparse vectors, CBOW menghasilkan dense vectors yang dapat menangkap hubungan semantik antar kata. Model ini berguna untuk analisis similarity dan clustering dokumen berdasarkan makna kontekstual.

In [12]:
# Install dan konfigurasi libraries untuk CBOW dengan versi kompatibel
import sys
import subprocess

def install_package(package):
    """Install package dengan handling error"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        return True
    except subprocess.CalledProcessError:
        return False

# Install packages dengan versi yang kompatibel
print("📦 Installing compatible packages...")

packages = [
    "numpy==1.24.4",  # Versi yang lebih kompatibel
    "scipy==1.10.1",   # Versi yang lebih stabil
    "gensim==4.3.0"    # Versi gensim yang stabil
]

for package in packages:
    print(f"   Installing {package}...")
    if install_package(package):
        print(f"   ✅ {package} installed successfully")
    else:
        print(f"   ⚠️ Failed to install {package}, trying alternative...")

# Import libraries dengan error handling yang lebih baik
print("\n🔄 Importing libraries...")
try:
    # Force reload modules jika sudah ada
    import importlib
    
    # Import dengan try-except individual
    import numpy as np
    print("   ✅ numpy imported")
    
    import pandas as pd
    print("   ✅ pandas imported")
    
    from sklearn.metrics.pairwise import cosine_similarity
    print("   ✅ sklearn imported")
    
    # Import gensim dengan handling khusus
    try:
        from gensim.models import Word2Vec
        print("   ✅ gensim Word2Vec imported")
        gensim_available = True
    except Exception as e:
        print(f"   ❌ gensim import error: {e}")
        print("   💡 Trying alternative import method...")
        
        try:
            # Alternative import method
            import gensim
            from gensim import models
            Word2Vec = models.Word2Vec
            print("   ✅ gensim imported via alternative method")
            gensim_available = True
        except Exception as e2:
            print(f"   ❌ Alternative gensim import failed: {e2}")
            gensim_available = False
            
    if gensim_available:
        print("🎉 Semua libraries berhasil diimport!")
    else:
        print("⚠️ Gensim tidak tersedia, akan skip analisis CBOW")
        
except Exception as e:
    print(f"❌ Critical import error: {e}")
    print("💡 Silakan restart kernel dan coba lagi")

# Lanjutkan jika gensim berhasil diimport
if 'gensim_available' in locals() and gensim_available:
    print("=== ANALISIS WORD EMBEDDINGS DENGAN CBOW ===\n")

    # Siapkan data untuk Word2Vec (perlu dalam format list of sentences)
    sentences = hasil_stemm  # hasil_stemm sudah berupa list of lists (sentences)

    print(f"📊 Data untuk training CBOW:")
    print(f"   • Total dokumen: {len(sentences)}")
    print(f"   • Rata-rata panjang dokumen: {np.mean([len(sent) for sent in sentences]):.1f} kata")
    print(f"   • Total unique words: {len(set([word for sent in sentences for word in sent]))}")

    # Training Word2Vec dengan arsitektur CBOW
    print(f"\n🤖 Training Word2Vec model dengan CBOW...")
    try:
        cbow_model = Word2Vec(
            sentences=sentences,
            vector_size=100,        # Dimensi embedding (dense vector)
            window=5,              # Ukuran context window
            min_count=2,           # Minimum frekuensi kata untuk dimasukkan
            sg=0,                  # sg=0 untuk CBOW, sg=1 untuk Skip-gram
            workers=4,             # Jumlah thread untuk training
            epochs=100,            # Jumlah iterasi training
            seed=42                # Untuk reproducibility
        )

        print(f"✅ Model CBOW berhasil dilatih!")
        print(f"   • Vocabulary size: {len(cbow_model.wv)} kata")
        print(f"   • Vector dimensions: {cbow_model.wv.vector_size}")
        print(f"   • Training epochs: {cbow_model.epochs}")

        # Analisis kata-kata terdekat
        print(f"\n🔍 ANALISIS SEMANTIC SIMILARITY:")

        # Pilih beberapa kata kunci untuk analisis
        sample_words = []
        vocab_words = list(cbow_model.wv.key_to_index.keys())

        # Ambil kata-kata yang relevan dengan domain manajemen
        target_keywords = ['manaj', 'perusaha', 'kinerja', 'strategi', 'analisis', 'peneliti', 'hasil', 'faktor']
        for keyword in target_keywords:
            # Cari kata yang mengandung keyword
            matching_words = [word for word in vocab_words if keyword in word]
            if matching_words:
                sample_words.append(matching_words[0])  # Ambil yang pertama

        # Tampilkan analisis similarity untuk beberapa kata
        for word in sample_words[:5]:  # Ambil 5 kata pertama
            if word in cbow_model.wv:
                try:
                    similar_words = cbow_model.wv.most_similar(word, topn=5)
                    print(f"\n📝 Kata mirip dengan '{word}':")
                    for similar_word, score in similar_words:
                        print(f"      {similar_word:15s} | Similarity: {score:.4f}")
                except Exception as e:
                    print(f"   Tidak bisa mencari kata mirip untuk '{word}': {e}")

        cbow_success = True

    except Exception as e:
        print(f"❌ Error training CBOW model: {e}")
        print("💡 Model tidak dapat dilatih, akan skip analisis lanjutan")
        cbow_success = False

    print(f"\n" + "="*60)

else:
    print("⚠️ GENSIM TIDAK TERSEDIA - SKIP ANALISIS CBOW")
    print("Analisis CBOW memerlukan library gensim yang tidak dapat diimport.")
    print("Silakan install ulang atau gunakan environment yang berbeda.")
    cbow_success = False

📦 Installing compatible packages...
   Installing numpy==1.24.4...
   ✅ numpy==1.24.4 installed successfully
   Installing scipy==1.10.1...
   ✅ numpy==1.24.4 installed successfully
   Installing scipy==1.10.1...
   ✅ scipy==1.10.1 installed successfully
   Installing gensim==4.3.0...
   ✅ scipy==1.10.1 installed successfully
   Installing gensim==4.3.0...
   ✅ gensim==4.3.0 installed successfully

🔄 Importing libraries...
   ✅ numpy imported
   ✅ pandas imported
   ✅ sklearn imported
   ✅ gensim Word2Vec imported
🎉 Semua libraries berhasil diimport!
=== ANALISIS WORD EMBEDDINGS DENGAN CBOW ===

📊 Data untuk training CBOW:
   • Total dokumen: 655
   • Rata-rata panjang dokumen: 145.5 kata
   • Total unique words: 4872

🤖 Training Word2Vec model dengan CBOW...
   ✅ gensim==4.3.0 installed successfully

🔄 Importing libraries...
   ✅ numpy imported
   ✅ pandas imported
   ✅ sklearn imported
   ✅ gensim Word2Vec imported
🎉 Semua libraries berhasil diimport!
=== ANALISIS WORD EMBEDDINGS DEN

In [13]:
# Lanjutkan document embeddings jika CBOW berhasil
if 'cbow_success' in locals() and cbow_success and 'cbow_model' in locals():
    # Membuat Document Embeddings dengan rata-rata Word Vectors
    print(f"🔢 PEMBUATAN DOCUMENT EMBEDDINGS")
    print(f"="*50)

    def create_document_embedding(tokens, model):
        """Membuat embedding dokumen dengan rata-rata word vectors"""
        vectors = []
        valid_words = 0
        
        for word in tokens:
            if word in model.wv:
                vectors.append(model.wv[word])
                valid_words += 1
        
        if vectors:
            # Rata-rata dari semua word vectors dalam dokumen
            doc_embedding = np.mean(vectors, axis=0)
            return doc_embedding, valid_words
        else:
            # Jika tidak ada kata yang valid, return zero vector
            return np.zeros(model.wv.vector_size), 0
else:
    print("⚠️ SKIP DOCUMENT EMBEDDINGS - CBOW MODEL TIDAK TERSEDIA")
    print("Document embeddings memerlukan model CBOW yang sudah dilatih.")
    
if 'cbow_success' in locals() and cbow_success and 'cbow_model' in locals():

    # Buat embeddings untuk semua dokumen
    print(f"\n📋 Membuat embeddings untuk {len(sentences)} dokumen...")

    document_embeddings = []
    valid_words_count = []
    doc_info = []

    for idx, tokens in enumerate(sentences):
        embedding, valid_count = create_document_embedding(tokens, cbow_model)
        document_embeddings.append(embedding)
        valid_words_count.append(valid_count)
        
        # Simpan informasi dokumen
        original_text = ' '.join(tokens[:10])  # 10 kata pertama sebagai preview
        doc_info.append({
            'doc_id': idx,
            'preview': original_text + ('...' if len(tokens) > 10 else ''),
            'total_words': len(tokens),
            'valid_words': valid_count,
            'coverage': valid_count / len(tokens) if len(tokens) > 0 else 0
        })

    # Convert ke numpy array
    document_embeddings = np.array(document_embeddings)

    print(f"✅ Document embeddings berhasil dibuat!")
    print(f"   • Shape: {document_embeddings.shape}")
    print(f"   • Rata-rata kata valid per dokumen: {np.mean(valid_words_count):.1f}")
    print(f"   • Coverage rata-rata: {np.mean([info['coverage'] for info in doc_info]):.3f}")

    # Hitung Document Similarity Matrix menggunakan Cosine Similarity
    print(f"\n🔗 Menghitung similarity antar dokumen...")
    doc_similarity_matrix = cosine_similarity(document_embeddings)

    print(f"✅ Similarity matrix berhasil dibuat!")
    print(f"   • Shape: {doc_similarity_matrix.shape}")
    print(f"   • Rata-rata similarity: {np.mean(doc_similarity_matrix[np.triu_indices_from(doc_similarity_matrix, k=1)]):.4f}")

    # Analisis dokumen paling mirip
    print(f"\n🎯 ANALISIS DOKUMEN PALING MIRIP:")
    print(f"="*45)

    # Cari pasangan dokumen dengan similarity tertinggi
    upper_triangle = np.triu_indices_from(doc_similarity_matrix, k=1)
    similarities = doc_similarity_matrix[upper_triangle]
    sorted_indices = np.argsort(similarities)[::-1]

    print(f"\n📊 Top 5 pasangan dokumen paling mirip:")
    for i in range(min(5, len(sorted_indices))):
        idx = sorted_indices[i]
        doc_i, doc_j = upper_triangle[0][idx], upper_triangle[1][idx]
        similarity_score = similarities[idx]
        
        print(f"\n{i+1}. Dokumen {doc_i} ↔ Dokumen {doc_j}")
        print(f"   📈 Similarity Score: {similarity_score:.4f}")
        print(f"   📝 Preview Doc {doc_i}: {doc_info[doc_i]['preview']}")
        print(f"   📝 Preview Doc {doc_j}: {doc_info[doc_j]['preview']}")

    print(f"\n" + "="*60)

🔢 PEMBUATAN DOCUMENT EMBEDDINGS

📋 Membuat embeddings untuk 655 dokumen...
✅ Document embeddings berhasil dibuat!
   • Shape: (655, 100)
   • Rata-rata kata valid per dokumen: 142.4
   • Coverage rata-rata: 0.971

🔗 Menghitung similarity antar dokumen...
✅ Similarity matrix berhasil dibuat!
   • Shape: (655, 655)
   • Rata-rata similarity: 0.2804

🎯 ANALISIS DOKUMEN PALING MIRIP:

📊 Top 5 pasangan dokumen paling mirip:

1. Dokumen 253 ↔ Dokumen 303
   📈 Similarity Score: 1.0000
   📝 Preview Doc 253: maulina hotim pengaruh pribadi mampu individu prestasi kerja karyawan dinas...
   📝 Preview Doc 303: maulina hotim pengaruh pribadi mampu individu prestasi kerja karyawan dinas...

2. Dokumen 331 ↔ Dokumen 647
   📈 Similarity Score: 0.9871
   📝 Preview Doc 331: teliti pt bank rakyat indonesia persero tbk cabang diponegoro surabaya...
   📝 Preview Doc 647: teliti judul pengaruh rotasi kerja motivasi kerja kerja karyawan puas...

3. Dokumen 376 ↔ Dokumen 483
   📈 Similarity Score: 0.9860
   📝

In [14]:
# EXPORT HASIL CBOW KE EXCEL DENGAN FORMAT PROFESIONAL
if 'cbow_success' in locals() and cbow_success and 'document_embeddings' in locals():
    print(f"💾 EXPORT HASIL CBOW KE EXCEL MULTI-SHEET")
    print(f"="*50)
    
    try:
        from openpyxl import Workbook
        from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
        from openpyxl.utils.dataframe import dataframe_to_rows
        from openpyxl.formatting.rule import ColorScaleRule
        
        # Buat workbook baru untuk CBOW
        wb_cbow = Workbook()
        
        # Definisi styles yang sama seperti TF-IDF
        header_font = Font(bold=True, color="FFFFFF", size=11)
        header_fill = PatternFill(start_color="2E8B57", end_color="2E8B57", fill_type="solid")  # Sea Green untuk CBOW
        header_alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
        
        data_font = Font(size=9)
        data_alignment_left = Alignment(horizontal="left", vertical="center")
        data_alignment_center = Alignment(horizontal="center", vertical="center")
        data_alignment_right = Alignment(horizontal="right", vertical="center")
        
        thin_border = Border(
            left=Side(style='thin'), right=Side(style='thin'),
            top=Side(style='thin'), bottom=Side(style='thin')
        )
        
        # Sheet 1: Penjelasan Konsep CBOW
        ws1 = wb_cbow.active
        ws1.title = "Penjelasan_CBOW"
        
        penjelasan_cbow = [
            ['Aspek', 'CBOW (Continuous Bag of Words)', 'Penjelasan Detail', 'Contoh Aplikasi'],
            ['Definisi', 'Neural network yang memprediksi kata target dari konteks', 'Menggunakan kata-kata di sekitar untuk memprediksi kata tengah', 'Prediksi kata hilang dalam kalimat'],
            ['Arsitektur', 'Input: context words → Hidden layer → Output: target word', 'Rata-rata word vectors sebagai input ke hidden layer', 'Context: [penelitian, ini, tentang] → Target: menganalisis'],
            ['Vector Output', 'Dense vectors (100 dimensi)', 'Representasi padat yang menangkap semantic similarity', 'Kata "analisis" dan "penelitian" memiliki vector yang mirip'],
            ['Training Process', 'Sliding window melewati corpus', 'Window bergeser untuk create training pairs', 'Window size=5: ambil 2 kata kiri-kanan sebagai context'],
            ['', '', '', ''],
            ['Perbedaan dengan TF-IDF', '', '', ''],
            ['Representasi', 'Dense vectors (padat)', 'TF-IDF: Sparse vectors (jarang)', 'CBOW: 100 dimensi, TF-IDF: 6475 dimensi'],
            ['Basis Perhitungan', 'Semantic context', 'TF-IDF: Statistical frequency', 'CBOW: makna, TF-IDF: frekuensi'],
            ['Similarity Measure', 'Cosine similarity antar dense vectors', 'TF-IDF: Dot product atau cosine', 'CBOW menangkap hubungan semantic lebih baik'],
            ['Interpretasi', 'Semantic relationships', 'TF-IDF: Term importance', 'CBOW: kata mirip secara makna'],
            ['', '', '', ''],
            ['Parameter Model CBOW', '', '', ''],
            [f'Vector Size', f'{cbow_model.wv.vector_size}', 'Dimensi embedding untuk setiap kata', 'Lebih tinggi = representasi lebih detail'],
            [f'Window Size', f'{cbow_model.window}', 'Jumlah kata konteks kiri-kanan', 'Window=5 berarti 5 kata sebelum + 5 setelah'],
            [f'Min Count', f'{cbow_model.min_count}', 'Frekuensi minimum kata untuk dimasukkan', 'Kata < 2x diabaikan untuk mengurangi noise'],
            [f'Training Epochs', f'{cbow_model.epochs}', 'Jumlah iterasi training', 'Lebih banyak epoch = learning lebih dalam'],
            [f'Total Vocabulary', f'{len(cbow_model.wv)}', 'Ukuran kosakata model', 'Kata-kata yang dipelajari oleh model'],
            ['', '', '', ''],
            ['Statistik Dataset', '', '', ''],
            [f'Total Dokumen', f'{len(sentences)}', 'Jumlah abstrak yang dianalisis', 'Corpus penelitian manajemen'],
            [f'Rata-rata Panjang Dokumen', f'{np.mean([len(sent) for sent in sentences]):.1f} kata', 'Panjang dokumen setelah preprocessing', 'Termasuk stemming dan stopword removal'],
            [f'Coverage Rata-rata', f'{np.mean([info["coverage"] for info in doc_info]):.3f}', 'Persentase kata yang dikenali model', 'Kata yang ada di vocabulary model'],
            [f'Similarity Rata-rata', f'{np.mean(doc_similarity_matrix[np.triu_indices_from(doc_similarity_matrix, k=1)]):.4f}', 'Rata-rata kesamaan antar dokumen', 'Berbasis semantic similarity'],
            [f'Max Similarity', f'{np.max(doc_similarity_matrix[np.triu_indices_from(doc_similarity_matrix, k=1)]):.4f}', 'Similarity tertinggi antar dokumen', 'Dokumen paling mirip secara semantic']
        ]
        
        for row in penjelasan_cbow:
            ws1.append(row)
        
        # Format sheet penjelasan CBOW
        for row_idx, row in enumerate(ws1.iter_rows(min_row=1, max_row=len(penjelasan_cbow)), 1):
            for col_idx, cell in enumerate(row):
                if row_idx == 1 or row_idx == 7 or row_idx == 13 or row_idx == 19:  # Headers
                    cell.font = Font(bold=True, size=11, color="FFFFFF")
                    cell.fill = PatternFill(start_color="2E8B57", end_color="2E8B57", fill_type="solid")
                elif row_idx in [6, 12, 18]:  # Empty rows
                    continue
                else:
                    cell.font = Font(size=10)
                
                cell.alignment = Alignment(horizontal="left", vertical="top", wrap_text=True)
                cell.border = thin_border
        
        # Set column widths
        ws1.column_dimensions['A'].width = 25
        ws1.column_dimensions['B'].width = 35
        ws1.column_dimensions['C'].width = 45
        ws1.column_dimensions['D'].width = 35
        
        # Sheet 2: Training Details
        ws2 = wb_cbow.create_sheet("Detail_Training")
        
        # Buat data training details
        training_details = []
        
        # Ambil sample dari vocabulary untuk menunjukkan word vectors
        sample_vocab = list(cbow_model.wv.key_to_index.keys())[:30]  # Top 30 kata
        
        for word in sample_vocab:
            word_vector = cbow_model.wv[word]
            frequency = cbow_model.wv.get_vecattr(word, 'count')
            vector_norm = np.linalg.norm(word_vector)
            
            # Cari kata-kata paling mirip
            try:
                similar_words = cbow_model.wv.most_similar(word, topn=3)
                top_similar = ', '.join([f"{w}({s:.3f})" for w, s in similar_words])
            except:
                top_similar = "N/A"
            
            training_details.append({
                'Kata': word,
                'Frekuensi': frequency,
                'Vector_Norm': round(vector_norm, 4),
                'Kata_Mirip_Top3': top_similar,
                'Dimensi_1': round(word_vector[0], 4),
                'Dimensi_2': round(word_vector[1], 4),
                'Dimensi_3': round(word_vector[2], 4),
                'Dimensi_4': round(word_vector[3], 4),
                'Dimensi_5': round(word_vector[4], 4)
            })
        
        training_df = pd.DataFrame(training_details)
        
        # Tulis training data
        for r in dataframe_to_rows(training_df, index=False, header=True):
            ws2.append(r)
        
        # Format header sheet 2
        for cell in ws2[1]:
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
        
        # Set column widths dan alignment
        ws2_widths = [15, 12, 15, 40, 12, 12, 12, 12, 12]
        ws2_alignments = [data_alignment_left, data_alignment_center, data_alignment_right,
                         data_alignment_left, data_alignment_right, data_alignment_right, 
                         data_alignment_right, data_alignment_right, data_alignment_right]
        
        for idx, (width, alignment) in enumerate(zip(ws2_widths, ws2_alignments)):
            col_letter = chr(65 + idx)
            ws2.column_dimensions[col_letter].width = width
            
            for row in ws2.iter_rows(min_row=2, min_col=idx+1, max_col=idx+1):
                for cell in row:
                    cell.font = data_font
                    cell.alignment = alignment
                    cell.border = thin_border
                    if idx in [1]:  # Frequency
                        cell.number_format = '0'
                    elif idx in [2, 4, 5, 6, 7, 8]:  # Vector values
                        cell.number_format = '0.0000'
        
        ws2.freeze_panes = "A2"
        ws2.auto_filter.ref = ws2.dimensions
        ws2.row_dimensions[1].height = 30
        
        # Sheet 3: Document Similarity Analysis
        ws3 = wb_cbow.create_sheet("Analisis_Similarity")
        
        # Buat data untuk top similar document pairs
        similar_pairs = []
        for i in range(min(50, len(sorted_indices))):
            idx = sorted_indices[i]
            doc_i, doc_j = upper_triangle[0][idx], upper_triangle[1][idx]
            similarity_score = similarities[idx]
            
            similar_pairs.append({
                'Ranking': i + 1,
                'ID_Dokumen_1': doc_i,
                'ID_Dokumen_2': doc_j,
                'Skor_Similarity': round(similarity_score, 6),
                'Preview_Dok_1': doc_info[doc_i]['preview'][:50] + "...",
                'Preview_Dok_2': doc_info[doc_j]['preview'][:50] + "...",
                'Panjang_Dok_1': doc_info[doc_i]['total_words'],
                'Panjang_Dok_2': doc_info[doc_j]['total_words'],
                'Coverage_1': round(doc_info[doc_i]['coverage'], 4),
                'Coverage_2': round(doc_info[doc_j]['coverage'], 4),
                'Kategori_Similarity': 'Sangat Tinggi' if similarity_score > 0.8 else 'Tinggi' if similarity_score > 0.6 else 'Sedang' if similarity_score > 0.4 else 'Rendah'
            })
        
        similarity_pairs_df = pd.DataFrame(similar_pairs)
        
        # Tulis similarity data
        for r in dataframe_to_rows(similarity_pairs_df, index=False, header=True):
            ws3.append(r)
        
        # Format header sheet 3
        for cell in ws3[1]:
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
        
        # Set column widths
        ws3_widths = [10, 15, 15, 15, 35, 35, 12, 12, 12, 12, 15]
        ws3_alignments = [data_alignment_center, data_alignment_center, data_alignment_center,
                         data_alignment_right, data_alignment_left, data_alignment_left,
                         data_alignment_center, data_alignment_center, data_alignment_right, 
                         data_alignment_right, data_alignment_center]
        
        for idx, (width, alignment) in enumerate(zip(ws3_widths, ws3_alignments)):
            col_letter = chr(65 + idx)
            ws3.column_dimensions[col_letter].width = width
            
            for row in ws3.iter_rows(min_row=2, min_col=idx+1, max_col=idx+1):
                for cell in row:
                    cell.font = data_font
                    cell.alignment = alignment
                    cell.border = thin_border
                    if idx in [3, 8, 9]:  # Similarity and coverage scores
                        cell.number_format = '0.0000'
                    elif idx in [6, 7]:  # Document lengths
                        cell.number_format = '0'
        
        # Conditional formatting untuk similarity score
        ws3.conditional_formatting.add(f"D2:D{ws3.max_row}", 
                                      ColorScaleRule(start_type='min', start_color='FFFFFF',
                                                   mid_type='percentile', mid_value=50, mid_color='98FB98',
                                                   end_type='max', end_color='228B22'))
        
        ws3.freeze_panes = "A2"
        ws3.auto_filter.ref = ws3.dimensions
        ws3.row_dimensions[1].height = 30
        
        # Sheet 4: Document Embeddings Summary  
        ws4 = wb_cbow.create_sheet("Ringkasan_Embeddings")
        
        # Buat summary data untuk dokumen embeddings
        doc_summary = []
        for idx, info in enumerate(doc_info[:100]):  # Top 100 dokumen
            embedding = document_embeddings[idx]
            
            doc_summary.append({
                'ID_Dokumen': info['doc_id'],
                'Preview_Dokumen': info['preview'],
                'Total_Kata': info['total_words'],
                'Kata_Valid': info['valid_words'], 
                'Coverage': round(info['coverage'], 4),
                'Embedding_Norm': round(np.linalg.norm(embedding), 4),
                'Rata_Embedding': round(np.mean(embedding), 6),
                'Std_Embedding': round(np.std(embedding), 6),
                'Min_Embedding': round(np.min(embedding), 6),
                'Max_Embedding': round(np.max(embedding), 6)
            })
        
        doc_summary_df = pd.DataFrame(doc_summary)
        
        # Tulis document summary data
        for r in dataframe_to_rows(doc_summary_df, index=False, header=True):
            ws4.append(r)
        
        # Format header sheet 4
        for cell in ws4[1]:
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
        
        # Set column widths
        ws4_widths = [12, 50, 12, 12, 12, 15, 15, 15, 15, 15]
        ws4_alignments = [data_alignment_center, data_alignment_left, data_alignment_center,
                         data_alignment_center, data_alignment_right, data_alignment_right,
                         data_alignment_right, data_alignment_right, data_alignment_right, data_alignment_right]
        
        for idx, (width, alignment) in enumerate(zip(ws4_widths, ws4_alignments)):
            col_letter = chr(65 + idx)
            ws4.column_dimensions[col_letter].width = width
            
            for row in ws4.iter_rows(min_row=2, min_col=idx+1, max_col=idx+1):
                for cell in row:
                    cell.font = data_font
                    cell.alignment = alignment
                    cell.border = thin_border
                    if idx in [2, 3]:  # Word counts
                        cell.number_format = '0'
                    elif idx in [4, 5, 6, 7, 8, 9]:  # Decimal values
                        if idx == 4:  # Coverage
                            cell.number_format = '0.0000'
                        else:  # Other metrics
                            cell.number_format = '0.000000'
        
        # Conditional formatting untuk coverage
        ws4.conditional_formatting.add(f"E2:E{ws4.max_row}", 
                                      ColorScaleRule(start_type='min', start_color='FFB6C1',
                                                   end_type='max', end_color='90EE90'))
        
        ws4.freeze_panes = "A2"
        ws4.auto_filter.ref = ws4.dimensions
        ws4.row_dimensions[1].height = 30
        
        # Sheet 5: Model Statistics dan Comparison
        ws5 = wb_cbow.create_sheet("Statistik_Model")
        
        # Buat comprehensive model statistics
        model_statistics = [
            ['Metrik Model', 'Nilai', 'Deskripsi', 'Interpretasi'],
            ['Parameter Arsitektur', '', '', ''],
            ['Vector Dimensions', cbow_model.wv.vector_size, 'Ukuran embedding untuk setiap kata', 'Lebih tinggi = representasi lebih kaya'],
            ['Vocabulary Size', len(cbow_model.wv), 'Jumlah kata unik yang dipelajari', 'Ukuran kosakata model'],
            ['Window Size', cbow_model.window, 'Jumlah kata konteks kiri-kanan', 'Context window untuk prediksi'],
            ['Min Count Threshold', cbow_model.min_count, 'Frekuensi minimum kata', 'Filter noise dan kata jarang'],
            ['Training Epochs', cbow_model.epochs, 'Jumlah iterasi pelatihan', 'Tingkat pembelajaran model'],
            ['', '', '', ''],
            ['Statistik Dataset', '', '', ''],
            ['Total Documents', len(sentences), 'Jumlah dokumen dalam corpus', 'Ukuran dataset training'],
            ['Average Doc Length', f"{np.mean([len(sent) for sent in sentences]):.1f}", 'Rata-rata panjang dokumen', 'Kata per dokumen setelah preprocessing'],
            ['Total Unique Words', len(set([word for sent in sentences for word in sent])), 'Kata unik dalam corpus', 'Kekayaan vocabulary corpus'],
            ['Average Valid Words/Doc', f"{np.mean(valid_words_count):.1f}", 'Kata valid rata-rata per dokumen', 'Kata yang dikenali model'],
            ['Average Coverage', f"{np.mean([info['coverage'] for info in doc_info]):.4f}", 'Persentase kata yang dikenali', 'Tingkat coverage model terhadap corpus'],
            ['', '', '', ''], 
            ['Analisis Similarity', '', '', ''],
            ['Average Similarity', f"{np.mean(doc_similarity_matrix[np.triu_indices_from(doc_similarity_matrix, k=1)]):.6f}", 'Rata-rata similarity antar dokumen', 'Tingkat kesamaan semantic corpus'],
            ['Max Similarity', f"{np.max(doc_similarity_matrix[np.triu_indices_from(doc_similarity_matrix, k=1)]):.6f}", 'Similarity tertinggi', 'Dokumen paling mirip'],
            ['Min Similarity', f"{np.min(doc_similarity_matrix[np.triu_indices_from(doc_similarity_matrix, k=1)]):.6f}", 'Similarity terendah', 'Dokumen paling berbeda'],
            ['Std Similarity', f"{np.std(doc_similarity_matrix[np.triu_indices_from(doc_similarity_matrix, k=1)]):.6f}", 'Standar deviasi similarity', 'Variasi kesamaan antar dokumen'],
            ['', '', '', ''],
            ['Perbandingan dengan TF-IDF', '', '', ''],
            ['Vector Type', 'Dense (padat)', 'TF-IDF: Sparse (jarang)', 'CBOW lebih compact'],
            ['Dimensionality', '100 dimensi', 'TF-IDF: ~6475 dimensi', 'CBOW lebih efisien storage'],
            ['Semantic Capture', 'Ya (context-based)', 'TF-IDF: Tidak (frequency-based)', 'CBOW menangkap makna'],
            ['Interpretability', 'Similarity semantic', 'TF-IDF: Term importance', 'Berbeda tujuan analisis'],
            ['Computational Cost', 'Training: Tinggi, Inference: Rendah', 'TF-IDF: Training: Rendah, Inference: Sedang', 'Trade-off waktu vs kualitas']
        ]
        
        for row in model_statistics:
            ws5.append(row)
        
        # Format sheet statistics
        for row_idx, row in enumerate(ws5.iter_rows(min_row=1, max_row=len(model_statistics)), 1):
            for col_idx, cell in enumerate(row):
                if row_idx in [1, 2, 9, 16, 21]:  # Main headers
                    cell.font = Font(bold=True, size=11, color="FFFFFF")
                    cell.fill = PatternFill(start_color="2E8B57", end_color="2E8B57", fill_type="solid")
                elif row_idx in [8, 15, 20]:  # Empty rows
                    continue
                else:
                    cell.font = Font(size=10)
                
                cell.alignment = Alignment(horizontal="left", vertical="top", wrap_text=True)
                cell.border = thin_border
        
        # Set column widths untuk statistics sheet
        ws5.column_dimensions['A'].width = 25
        ws5.column_dimensions['B'].width = 20
        ws5.column_dimensions['C'].width = 35
        ws5.column_dimensions['D'].width = 40
        
        # Simpan Excel file untuk CBOW
        wb_cbow.save('CBOW_Analysis_Formatted.xlsx')
        
        print("✅ Excel CBOW berhasil dibuat dengan 5 sheet komprehensif:")
        print("   📊 CBOW_Analysis_Formatted.xlsx - Analisis lengkap CBOW")
        print("      • Sheet 1: Penjelasan_CBOW - Konsep dan teori CBOW vs TF-IDF")
        print("      • Sheet 2: Detail_Training - Sample word vectors dan similarity")
        print("      • Sheet 3: Analisis_Similarity - Top pasangan dokumen mirip")
        print("      • Sheet 4: Ringkasan_Embeddings - Summary document embeddings")
        print("      • Sheet 5: Statistik_Model - Comprehensive model statistics")
        print(f"\n📈 CBOW Analysis Summary:")
        print(f"   • Total documents analyzed: {len(doc_info):,}")
        print(f"   • Vocabulary size: {len(cbow_model.wv):,}")
        print(f"   • Vector dimensions: {cbow_model.wv.vector_size}")
        print(f"   • Average document similarity: {np.mean(doc_similarity_matrix[np.triu_indices_from(doc_similarity_matrix, k=1)]):.6f}")
        print(f"   • Top document similarity: {np.max(doc_similarity_matrix[np.triu_indices_from(doc_similarity_matrix, k=1)]):.6f}")
        
        cbow_excel_created = True
        
    except ImportError:
        print("⚠️ openpyxl not available - creating CSV files instead:")
        
        # Fallback ke CSV jika openpyxl tidak tersedia
        doc_df = pd.DataFrame(doc_info)
        doc_df['embedding_vector'] = [list(emb) for emb in document_embeddings]
        doc_df.to_csv('CBOW_Document_Info.csv', index=False, encoding='utf-8-sig')
        
        similarity_df = pd.DataFrame(
            doc_similarity_matrix,
            columns=[f'Doc_{i}' for i in range(len(doc_similarity_matrix))],
            index=[f'Doc_{i}' for i in range(len(doc_similarity_matrix))]
        )
        similarity_df.to_csv('CBOW_Document_Similarity_Matrix.csv', encoding='utf-8-sig')
        
        print("   📄 CBOW_Document_Info.csv - Document information with embeddings")
        print("   📄 CBOW_Document_Similarity_Matrix.csv - Similarity matrix")
        cbow_excel_created = False

else:
    print("⚠️ SKIP EXPORT CBOW - Data tidak tersedia")
    print("Export memerlukan model CBOW dan embeddings yang sudah berhasil dibuat.")
    print("Silakan perbaiki masalah gensim terlebih dahulu.")

print(f"\n" + "="*60)

💾 EXPORT HASIL CBOW KE EXCEL MULTI-SHEET
✅ Excel CBOW berhasil dibuat dengan 5 sheet komprehensif:
   📊 CBOW_Analysis_Formatted.xlsx - Analisis lengkap CBOW
      • Sheet 1: Penjelasan_CBOW - Konsep dan teori CBOW vs TF-IDF
      • Sheet 2: Detail_Training - Sample word vectors dan similarity
      • Sheet 3: Analisis_Similarity - Top pasangan dokumen mirip
      • Sheet 4: Ringkasan_Embeddings - Summary document embeddings
      • Sheet 5: Statistik_Model - Comprehensive model statistics

📈 CBOW Analysis Summary:
   • Total documents analyzed: 655
   • Vocabulary size: 2,818
   • Vector dimensions: 100
   • Average document similarity: 0.280438
   • Top document similarity: 1.000000

✅ Excel CBOW berhasil dibuat dengan 5 sheet komprehensif:
   📊 CBOW_Analysis_Formatted.xlsx - Analisis lengkap CBOW
      • Sheet 1: Penjelasan_CBOW - Konsep dan teori CBOW vs TF-IDF
      • Sheet 2: Detail_Training - Sample word vectors dan similarity
      • Sheet 3: Analisis_Similarity - Top pasangan d

### **6. Perbandingan Hasil: TF-IDF vs CBOW**

Setelah menjalankan analisis **TF-IDF** dan **CBOW**, kita dapat membandingkan kedua pendekatan ini:

#### **🔍 Perbedaan Pendekatan:**

| **Aspek** | **TF-IDF** | **CBOW** |
|-----------|------------|----------|
| **Jenis Vector** | Sparse (jarang) | Dense (padat) |
| **Dimensi** | Sebesar vocabulary (6,475) | Fixed (100 dimensi) |
| **Basis Perhitungan** | Frekuensi statistik | Semantic context |
| **Interpretasi** | Importance berdasarkan frekuensi | Similarity berdasarkan konteks |
| **Ukuran File** | Besar untuk sparse matrix | Lebih kecil untuk dense vectors |

#### **📊 Output yang Dihasilkan:**

**TF-IDF Files:**
- `TF_IDF_Analysis_Formatted.xlsx` - Analisis lengkap dalam Excel
- Matrix TF-IDF untuk klasifikasi dan pencarian

**CBOW Files:**
- `CBOW_Document_Info.csv` - Informasi dokumen dengan embeddings
- `CBOW_Document_Similarity_Matrix.csv` - Matrix similarity semantik  
- `CBOW_Top_Similar_Pairs.csv` - Pasangan dokumen paling mirip
- `CBOW_Word_Embeddings.csv` - Word vectors untuk vocabulary
- `CBOW_Model_Statistics.csv` - Statistik model CBOW

#### **💡 Aplikasi Praktis:**

- **TF-IDF**: Cocok untuk information retrieval, klasifikasi teks, keyword extraction
- **CBOW**: Cocok untuk analisis semantic similarity, clustering berdasarkan makna, recommendation system

Kedua pendekatan ini melengkapi satu sama lain dan dapat digunakan bersamaan tergantung pada tujuan analisis yang diinginkan.

### **6. Menghitung dan Menyimpan File Hasil TF-IDF**

In [11]:
# TF-IDF Analysis - Complete Implementation with Indonesian Labels
import numpy as np
import pandas as pd
import math

# Siapkan data TF-IDF
feature_names = vectorizer.get_feature_names_out()
tfidf_matrix = x.toarray()
judul_jurnal = result_df['judul'].tolist()
idf_scores = vectorizer.idf_

# Buat TF-IDF table dengan kolom yang diminta
tfidf_results = []
for doc_idx in range(len(judul_jurnal)):
    jurnal_name = judul_jurnal[doc_idx][:80] + "..." if len(judul_jurnal[doc_idx]) > 80 else judul_jurnal[doc_idx]
    doc_tfidf = tfidf_matrix[doc_idx]
    
    for term_idx, tfidf_score in enumerate(doc_tfidf):
        if tfidf_score > 0:  # Hanya term yang muncul di dokumen
            term = feature_names[term_idx]
            idf_score = idf_scores[term_idx]
            
            tfidf_results.append({
                'Term': term,
                'Nama_Jurnal': jurnal_name,
                'IDF': round(idf_score, 4),
                'TF_IDF': round(tfidf_score, 6)
            })

# Buat DataFrame TF-IDF
tfidf_df = pd.DataFrame(tfidf_results)
tfidf_df = tfidf_df.sort_values('TF_IDF', ascending=False)

# Tambahkan nomor urut untuk kemudahan baca
tfidf_df['No'] = range(1, len(tfidf_df) + 1)
tfidf_df = tfidf_df[['No', 'Term', 'Nama_Jurnal', 'IDF', 'TF_IDF']]

# Buat ringkasan per term
term_stats = tfidf_df.groupby('Term').agg({
    'IDF': 'first',
    'TF_IDF': ['count', 'mean', 'max']
}).round(6)
term_stats.columns = ['IDF', 'Frekuensi_Muncul', 'Rata_TF_IDF', 'Max_TF_IDF']
term_stats = term_stats.reset_index().sort_values('Rata_TF_IDF', ascending=False)
term_stats['Peringkat'] = range(1, len(term_stats) + 1)
term_stats = term_stats[['Peringkat', 'Term', 'IDF', 'Frekuensi_Muncul', 'Rata_TF_IDF', 'Max_TF_IDF']]

# Coba buat file Excel dengan formatting profesional
try:
    from openpyxl import Workbook
    from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
    from openpyxl.utils.dataframe import dataframe_to_rows
    from openpyxl.formatting.rule import ColorScaleRule
    
    # Buat workbook baru
    wb = Workbook()
    
    # Definisi styles
    header_font = Font(bold=True, color="FFFFFF", size=11)
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    header_alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
    
    data_font = Font(size=9)
    data_alignment_left = Alignment(horizontal="left", vertical="center")
    data_alignment_center = Alignment(horizontal="center", vertical="center")
    data_alignment_right = Alignment(horizontal="right", vertical="center")
    
    thin_border = Border(
        left=Side(style='thin'), right=Side(style='thin'),
        top=Side(style='thin'), bottom=Side(style='thin')
    )
    
    # Sheet 1: Penjelasan Rumus (Formula Explanation)
    ws1 = wb.active
    ws1.title = "Penjelasan_Rumus"
    
    # Penjelasan rumus dalam bahasa Indonesia
    penjelasan_rumus = [
        ['Metrik', 'Rumus', 'Penjelasan', 'Contoh'],
        ['TF (Term Frequency)', 'TF = jumlah_term / total_kata_dalam_dokumen', 'Frekuensi kemunculan term dalam dokumen relatif terhadap panjang dokumen', 'Jika "analisis" muncul 3 kali dalam dokumen 100 kata: TF = 3/100 = 0.03'],
        ['IDF (Inverse Document Frequency)', 'IDF = log(total_dokumen / dokumen_yang_mengandung_term)', 'Tingkat kepentingan term di seluruh koleksi dokumen', 'Jika "analisis" muncul di 50 dari 1000 dokumen: IDF = log(1000/50) = 2.996'],
        ['TF-IDF', 'TF-IDF = TF × IDF', 'Gabungan skor kepentingan dan frekuensi', 'TF-IDF = 0.03 × 2.996 = 0.08988'],
        ['', '', '', ''],
        ['Panduan Interpretasi', '', '', ''],
        ['TF-IDF Tinggi (>0.1)', 'Term sering muncul di dokumen DAN jarang di koleksi', 'Term yang sangat karakteristik untuk dokumen', 'Istilah teknis spesifik penelitian'],
        ['TF-IDF Sedang (0.01-0.1)', 'Term yang cukup penting', 'Relevan tapi tidak unik untuk dokumen', 'Istilah penelitian umum'],
        ['TF-IDF Rendah (<0.01)', 'Term umum atau jarang muncul', 'Kurang karakteristik untuk dokumen', 'Kata umum, stopwords'],
        ['', '', '', ''],
        ['Statistik Data Penelitian', '', '', ''],
        [f'Total Dokumen Dianalisis', str(len(judul_jurnal)), 'Jumlah abstrak penelitian yang diproses', ''],
        [f'Term Unik Ditemukan', str(len(feature_names)), 'Ukuran kosakata setelah preprocessing', ''],
        [f'Total Pasangan Term-Dokumen', str(len(tfidf_df)), 'Nilai TF-IDF yang tidak nol', ''],
        [f'Rata-rata Skor TF-IDF', f"{tfidf_df['TF_IDF'].mean():.6f}", 'Rata-rata kepentingan semua term', ''],
        [f'Skor TF-IDF Tertinggi', f"{tfidf_df['TF_IDF'].max():.6f}", 'Pasangan term-dokumen paling karakteristik', ''],
        ['', '', '', ''],
        ['Tahapan Preprocessing Teks', '', '', ''],
        ['1. Pembersihan Teks', 'Lowercasing, hapus angka & tanda baca', 'Normalisasi teks untuk konsistensi', 'ANALISIS -> analisis'],
        ['2. Tokenisasi', 'Pemecahan kalimat menjadi kata', 'Memisahkan teks menjadi unit kata', '"data mining" -> ["data", "mining"]'],
        ['3. Stopwords Removal', 'Hapus kata umum (dan, atau, di, dll)', 'Fokus pada kata bermakna', '["penelitian", "dan", "data"] -> ["penelitian", "data"]'],
        ['4. Stemming', 'Ubah kata ke bentuk dasar', 'Mengurangi variasi bentuk kata', 'penelitian -> teliti, menganalisis -> analisis']
    ]
    
    for row in penjelasan_rumus:
        ws1.append(row)
    
    # Format sheet penjelasan
    for row_idx, row in enumerate(ws1.iter_rows(min_row=1, max_row=len(penjelasan_rumus)), 1):
        for col_idx, cell in enumerate(row):
            if row_idx == 1 or row_idx == 6 or row_idx == 11 or row_idx == 17:  # Headers
                cell.font = Font(bold=True, size=11, color="FFFFFF")
                cell.fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
            elif row_idx in [5, 10, 16]:  # Empty rows
                continue
            else:
                cell.font = Font(size=10)
                if col_idx == 1:  # Formula column
                    cell.font = Font(size=9, name="Courier New")
            
            cell.alignment = Alignment(horizontal="left", vertical="top", wrap_text=True)
            cell.border = thin_border
    
    # Set column widths untuk penjelasan sheet
    ws1.column_dimensions['A'].width = 30
    ws1.column_dimensions['B'].width = 45
    ws1.column_dimensions['C'].width = 40
    ws1.column_dimensions['D'].width = 45
    
    # Sheet 2: Perhitungan TF (TF Calculation Details)
    ws2 = wb.create_sheet("Perhitungan_TF")
    
    # Buat data TF calculation untuk beberapa dokumen pertama
    tf_details = []
    for doc_idx in range(min(15, len(judul_jurnal))):  # Ambil 15 dokumen pertama
        jurnal_name = judul_jurnal[doc_idx][:60] + "..." if len(judul_jurnal[doc_idx]) > 60 else judul_jurnal[doc_idx]
        doc_tfidf = tfidf_matrix[doc_idx]
        
        # Hitung total words dalam dokumen
        doc_words = stemmed_texts[doc_idx].split()
        total_words = len(doc_words)
        word_counts = {}
        for word in doc_words:
            word_counts[word] = word_counts.get(word, 0) + 1
        
        # Ambil top 8 terms untuk dokumen ini
        doc_terms = [(i, score) for i, score in enumerate(doc_tfidf) if score > 0]
        doc_terms.sort(key=lambda x: x[1], reverse=True)
        
        for term_idx, tfidf_score in doc_terms[:8]:  # Top 8 terms
            term = feature_names[term_idx]
            term_frequency = word_counts.get(term, 0)
            tf_score = term_frequency / total_words if total_words > 0 else 0
            
            tf_details.append({
                'ID_Dokumen': doc_idx + 1,
                'Nama_Jurnal': jurnal_name,
                'Term': term,
                'Jumlah_Term': term_frequency,
                'Total_Kata': total_words,
                'Skor_TF': round(tf_score, 6),
                'Rumus': f"{term_frequency}/{total_words}"
            })
    
    tf_df = pd.DataFrame(tf_details)
    
    # Tulis TF data
    for r in dataframe_to_rows(tf_df, index=False, header=True):
        ws2.append(r)
    
    # Format header sheet 2
    for cell in ws2[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
        cell.border = thin_border
    
    # Set column widths untuk TF sheet
    ws2_widths = [12, 45, 20, 12, 12, 15, 15]
    ws2_alignments = [data_alignment_center, data_alignment_left, data_alignment_left,
                     data_alignment_center, data_alignment_center, data_alignment_right, data_alignment_center]
    
    for idx, (width, alignment) in enumerate(zip(ws2_widths, ws2_alignments)):
        col_letter = chr(65 + idx)
        ws2.column_dimensions[col_letter].width = width
        
        for row in ws2.iter_rows(min_row=2, min_col=idx+1, max_col=idx+1):
            for cell in row:
                cell.font = data_font
                cell.alignment = alignment
                cell.border = thin_border
                if idx in [3, 4]:  # Count columns
                    cell.number_format = '0'
                elif idx == 5:  # TF_Score
                    cell.number_format = '0.000000'
    
    # Conditional formatting untuk TF Score
    ws2.conditional_formatting.add(f"F2:F{ws2.max_row}", 
                                  ColorScaleRule(start_type='min', start_color='FFFFFF',
                                               end_type='max', end_color='FFE699'))
    
    ws2.freeze_panes = "A2"
    ws2.auto_filter.ref = ws2.dimensions
    ws2.row_dimensions[1].height = 30
    
    # Sheet 3: Perhitungan IDF (IDF Calculation Details)
    ws3 = wb.create_sheet("Perhitungan_IDF")
    
    # Buat data IDF calculation
    idf_details = []
    total_documents = len(judul_jurnal)
    
    # Ambil top 50 terms berdasarkan rata-rata TF-IDF
    top_terms = term_stats.head(50)
    
    for _, row in top_terms.iterrows():
        term = row['Term']
        term_idx = list(feature_names).index(term)
        idf_score = idf_scores[term_idx]
        
        # Hitung berapa dokumen yang mengandung term ini
        docs_containing_term = sum(1 for doc_tfidf in tfidf_matrix if doc_tfidf[term_idx] > 0)
        calculated_idf = math.log(total_documents / docs_containing_term) if docs_containing_term > 0 else 0
        
        idf_details.append({
            'Term': term,
            'Total_Dokumen': total_documents,
            'Dokumen_Mengandung_Term': docs_containing_term,
            'Skor_IDF': round(idf_score, 6),
            'IDF_Terhitung': round(calculated_idf, 6),
            'Rumus': f"log({total_documents}/{docs_containing_term})",
            'Persentase_Frekuensi': round((docs_containing_term/total_documents)*100, 2)
        })
    
    idf_df = pd.DataFrame(idf_details)
    
    # Tulis IDF data
    for r in dataframe_to_rows(idf_df, index=False, header=True):
        ws3.append(r)
    
    # Format header sheet 3
    for cell in ws3[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
        cell.border = thin_border
    
    # Set column widths untuk IDF sheet
    ws3_widths = [20, 15, 20, 15, 15, 18, 15]
    ws3_alignments = [data_alignment_left, data_alignment_center, data_alignment_center,
                     data_alignment_right, data_alignment_right, data_alignment_center, data_alignment_right]
    
    for idx, (width, alignment) in enumerate(zip(ws3_widths, ws3_alignments)):
        col_letter = chr(65 + idx)
        ws3.column_dimensions[col_letter].width = width
        
        for row in ws3.iter_rows(min_row=2, min_col=idx+1, max_col=idx+1):
            for cell in row:
                cell.font = data_font
                cell.alignment = alignment
                cell.border = thin_border
                if idx in [1, 2]:  # Document counts
                    cell.number_format = '0'
                elif idx in [3, 4, 6]:  # IDF scores and percentage
                    if idx == 6:  # Percentage
                        cell.number_format = '0.00%'
                    else:  # IDF scores
                        cell.number_format = '0.000000'
    
    # Conditional formatting untuk frequency percentage
    ws3.conditional_formatting.add(f"G2:G{ws3.max_row}", 
                                  ColorScaleRule(start_type='max', start_color='FF9999',
                                               end_type='min', end_color='99FF99'))
    
    ws3.freeze_panes = "A2"
    ws3.auto_filter.ref = ws3.dimensions
    ws3.row_dimensions[1].height = 30
    
    # Sheet 4: Tabel TF-IDF (Final Results)
    ws4 = wb.create_sheet("Tabel_TF_IDF")
    
    # Tulis data TF-IDF hasil akhir
    for r in dataframe_to_rows(tfidf_df, index=False, header=True):
        ws4.append(r)
    
    # Format header sheet 4
    for cell in ws4[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
        cell.border = thin_border
    
    # Set column widths dan format data
    ws4_widths = [8, 25, 70, 12, 15]
    ws4_alignments = [data_alignment_center, data_alignment_left, data_alignment_left, 
                     data_alignment_right, data_alignment_right]
    
    for idx, (width, alignment) in enumerate(zip(ws4_widths, ws4_alignments)):
        col_letter = chr(65 + idx)
        ws4.column_dimensions[col_letter].width = width
        
        # Format data cells
        for row in ws4.iter_rows(min_row=2, min_col=idx+1, max_col=idx+1):
            for cell in row:
                cell.font = data_font
                cell.alignment = alignment
                cell.border = thin_border
                if idx in [3, 4]:  # IDF dan TF_IDF columns
                    cell.number_format = '0.000000'
    
    # Conditional formatting untuk TF_IDF
    ws4.conditional_formatting.add(f"E2:E{ws4.max_row}", 
                                  ColorScaleRule(start_type='min', start_color='FFFFFF',
                                               mid_type='percentile', mid_value=50, mid_color='FFE699',
                                               end_type='max', end_color='FF9999'))
    
    # Freeze panes dan filter
    ws4.freeze_panes = "A2"
    ws4.auto_filter.ref = ws4.dimensions
    ws4.row_dimensions[1].height = 30
    
    # Sheet 5: Statistik Term (Term Statistics)
    ws5 = wb.create_sheet("Statistik_Term")
    
    # Tulis data ringkasan term statistics
    for r in dataframe_to_rows(term_stats, index=False, header=True):
        ws5.append(r)
    
    # Format header sheet 5
    for cell in ws5[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
        cell.border = thin_border
    
    # Set column widths dan format untuk sheet 5
    ws5_widths = [10, 25, 12, 15, 15, 15]
    ws5_alignments = [data_alignment_center, data_alignment_left, data_alignment_right,
                     data_alignment_center, data_alignment_right, data_alignment_right]
    
    for idx, (width, alignment) in enumerate(zip(ws5_widths, ws5_alignments)):
        col_letter = chr(65 + idx)
        ws5.column_dimensions[col_letter].width = width
        
        # Format data cells
        for row in ws5.iter_rows(min_row=2, min_col=idx+1, max_col=idx+1):
            for cell in row:
                cell.font = data_font
                cell.alignment = alignment
                cell.border = thin_border
                if idx in [2, 4, 5]:  # Numerical columns
                    cell.number_format = '0.000000'
                elif idx == 3:  # Frekuensi column
                    cell.number_format = '0'
    
    # Conditional formatting untuk rata TF_IDF
    ws5.conditional_formatting.add(f"E2:E{ws5.max_row}", 
                                  ColorScaleRule(start_type='min', start_color='E8F5E8',
                                               end_type='max', end_color='4CAF50'))
    
    # Freeze panes dan filter untuk sheet 5
    ws5.freeze_panes = "A2"
    ws5.auto_filter.ref = ws5.dimensions
    ws5.row_dimensions[1].height = 30
    
    # Simpan Excel
    wb.save('TF_IDF_Analysis_Formatted.xlsx')
    
    print("✅ Excel file created with comprehensive TF-IDF analysis:")
    print("   📊 TF_IDF_Analysis_Formatted.xlsx - Analisis lengkap dengan 5 sheet terurut")
    print("      • Sheet 1: Penjelasan_Rumus - Panduan lengkap rumus dan interpretasi (Bahasa Indonesia)")
    print("      • Sheet 2: Perhitungan_TF - Detail perhitungan Term Frequency")
    print("      • Sheet 3: Perhitungan_IDF - Detail perhitungan Inverse Document Frequency")
    print("      • Sheet 4: Tabel_TF_IDF - Hasil akhir dengan conditional formatting")
    print("      • Sheet 5: Statistik_Term - Ringkasan statistik dengan peringkat")
    excel_created = True
    
except ImportError:
    print("⚠️ openpyxl not available - creating CSV files instead:")
    
    # Simpan ke CSV dengan formatting yang rapi
    tfidf_df.to_csv('TF_IDF_Table.csv', index=False, encoding='utf-8-sig', float_format='%.6f')
    term_stats.to_csv('Term_Statistics.csv', index=False, encoding='utf-8-sig', float_format='%.6f')
    
    print("   📄 TF_IDF_Table.csv - Main TF-IDF data with numbering")
    print("   📄 Term_Statistics.csv - Term summary with rankings")
    excel_created = False

print(f"\n📈 Analysis Summary:")
print(f"   • Total term-document pairs: {len(tfidf_df):,}")
print(f"   • Unique terms: {tfidf_df['Term'].nunique():,}")
print(f"   • Documents analyzed: {tfidf_df['Nama_Jurnal'].nunique():,}")
print(f"   • Top TF-IDF score: {tfidf_df['TF_IDF'].max():.6f}")
print(f"   • Average TF-IDF score: {tfidf_df['TF_IDF'].mean():.6f}")

if 'excel_created' in locals() and excel_created:
    print(f"\n💡 Excel file contains all data with professional formatting - CSV files not needed!")
else:
    print(f"\n💡 Use CSV files for data analysis or install openpyxl for better Excel formatting.")

✅ Excel file created with comprehensive TF-IDF analysis:
   📊 TF_IDF_Analysis_Formatted.xlsx - Analisis lengkap dengan 5 sheet terurut
      • Sheet 1: Penjelasan_Rumus - Panduan lengkap rumus dan interpretasi (Bahasa Indonesia)
      • Sheet 2: Perhitungan_TF - Detail perhitungan Term Frequency
      • Sheet 3: Perhitungan_IDF - Detail perhitungan Inverse Document Frequency
      • Sheet 4: Tabel_TF_IDF - Hasil akhir dengan conditional formatting
      • Sheet 5: Statistik_Term - Ringkasan statistik dengan peringkat

📈 Analysis Summary:
   • Total term-document pairs: 40,184
   • Unique terms: 4,850
   • Documents analyzed: 649
   • Top TF-IDF score: 0.823313
   • Average TF-IDF score: 0.084527

💡 Excel file contains all data with professional formatting - CSV files not needed!
